# Домашнее задание 3 [10 баллов]


# До 27.03.18 23:59

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная

### Предварительная обработка данных [2 балла]

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [2]:
# прочитаем файлы
female = open('female.txt', 'r').read().split('\n')
male = open('male.txt', 'r').read().split('\n')
print(len(female), len(male))

5001 2943


#### Удаляем неоднозначные имена

In [3]:
# найдем имена, которые одинаковы для двух списков
result = set(female) & set(male)

In [4]:
# удалим их!
female = list(set(female) - result)
male = list(set(male) - result)

In [5]:
print(len(female), len(male))

4636 2578


In [6]:
print(male[:100])

['Sansone', 'Rolland', 'Morley', 'Keefe', 'Howard', 'Pierre', 'Bealle', 'Fidel', 'Benton', 'Gallagher', 'Jakob', 'Wyatt', 'Padraig', 'Bryan', 'Agamemnon', 'Sayer', 'Hunter', 'Redford', 'Lawrence', 'Parker', 'Tad', 'Spense', 'Barde', 'Raynor', 'Forrest', 'Che', 'Antoni', 'Torrance', 'Lenard', 'Gibb', 'Niels', 'Rand', 'Neddy', 'Marten', 'Tann', 'Weber', 'Esau', 'Kenn', 'Corwin', 'Derby', 'John', 'Tremaine', 'Erek', 'Griffith', 'Haydon', 'Ellwood', 'Hamlen', 'Thaxter', 'Matt', 'Barclay', 'Sheppard', 'Jefferson', 'Leonidas', 'Hiram', 'Piet', 'Janos', 'Temp', 'Barnabas', 'Batholomew', 'Kendal', 'Wait', 'Remus', 'Forest', 'Samson', 'Rod', 'Arvie', 'Fernando', 'Ulrich', 'Nate', 'Rodolph', 'Alfonse', 'Barty', 'Briggs', 'Spence', 'Ric', 'Luis', 'Job', 'Gavin', 'Wyn', 'Fremont', 'Silvan', 'Nathanael', 'Jermayne', 'Will', 'Aguinaldo', 'Bryce', 'Parry', 'Cliff', 'Nevin', 'Vaclav', 'Archon', 'Rutherford', 'Rice', 'Xymenes', 'Berchtold', 'Maynord', 'Si', 'Orville', 'Mitchel', 'Clive']


#### Балансируем выборки

In [7]:
# oversampling!
missing = len(female) - len(male)
male += male[:missing]

In [8]:
# shuffling
import random
random.shuffle(male)

In [9]:
print(len(female), len(male))

4636 4636


In [10]:
boundary = int(len(female) * 0.8)
x_train = female[:boundary] + male[:boundary]
x_test = female[boundary:] + male[boundary:]
y_train = ['female'] * (boundary ) + ['male'] * (boundary)
y_test = ['female'] * (len(x_test)//2) + ['male'] * (len(x_test)//2)
print(len(x_train), len(x_test), len(y_train), len(y_test))

7416 1856 7416 1856


In [11]:
import pandas as pd
import numpy as np
index = random.sample(range(len(x_train)), len(x_train))
df = pd.DataFrame({'name': x_train, 'class': y_train})
df.head()

,class,name
0,female,Ardys
1,female,Gabrielle
2,female,Sukey
3,female,Koo
4,female,Pauli


In [12]:
# перемешаем
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,class,name
0,female,Vitia
1,female,Cynthia
2,female,Cybel
3,male,Terrel
4,female,Crystal


##  Часть 2. Базовая 

### Базовый метод классификации [3 балла]

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

Обучим три бейзлайновые модели на символьных 2, 3, и 4-граммах.

In [73]:
# биграммы
ppl = Pipeline(
    [('vect', CountVectorizer(ngram_range=(2, 2), analyzer='char')),
    ('nb', MultinomialNB())]
)
ppl.fit(x_train, y=y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [74]:
labels2 = ppl.predict(x_test)
print(classification_report(y_test, labels2))
print(accuracy_score(y_test, labels2))

             precision    recall  f1-score   support

     female       0.74      0.76      0.75       928
       male       0.76      0.74      0.75       928

avg / total       0.75      0.75      0.75      1856

0.748922413793


In [75]:
# триграммы
ppl = Pipeline(
    [('vect', CountVectorizer(ngram_range=(3, 3), analyzer='char')),
    ('nb', MultinomialNB())]
)
ppl.fit(x_train, y=y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(3, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [76]:
labels3 = ppl.predict(x_test)
print(classification_report(y_test, labels3))
print(accuracy_score(y_test, labels3))

             precision    recall  f1-score   support

     female       0.79      0.83      0.81       928
       male       0.82      0.78      0.80       928

avg / total       0.81      0.81      0.81      1856

0.806573275862


In [77]:
# квадриграммы
ppl = Pipeline(
    [('vect', CountVectorizer(ngram_range=(4, 4), analyzer='char')),
    ('nb', MultinomialNB())]
)
ppl.fit(x_train, y=y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(4, 4), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [78]:
labels4 = ppl.predict(x_test)
print(classification_report(y_test, labels4))
print(accuracy_score(y_test, labels4))

             precision    recall  f1-score   support

     female       0.81      0.88      0.84       928
       male       0.87      0.79      0.83       928

avg / total       0.84      0.84      0.84      1856

0.836745689655


Как мы видим, чем больше n, тем выше значение всех метрик.
Попробуем, из любопытства, n=5.

In [79]:
# пентаграммы (извините)
ppl = Pipeline(
    [('vect', CountVectorizer(ngram_range=(5, 5), analyzer='char')),
    ('nb', MultinomialNB())]
)
ppl.fit(x_train, y=y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(5, 5), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [80]:
labels = ppl.predict(x_test)
print(classification_report(y_test, labels))
print(accuracy_score(y_test, labels))

             precision    recall  f1-score   support

     female       0.73      0.93      0.82       928
       male       0.90      0.66      0.77       928

avg / total       0.82      0.80      0.79      1856

0.796336206897


При n=5 качество перестало расти.
Теперь проанализируем ошибки для лучшей модели (4-граммы).

In [81]:
print('name | true | predicted')
for i, name in enumerate(x_test):
    if y_test[i] != labels4[i]:
        print(name, ' | ' + y_test[i], ' | ' + labels4[i])

name | true | predicted
Agnola  | female  | male
Lesli  | female  | male
Raven  | female  | male
Sorcha  | female  | male
Joice  | female  | male
Wenda  | female  | male
Kailey  | female  | male
Martha  | female  | male
Alaine  | female  | male
Emanuela  | female  | male
Constantia  | female  | male
Phillis  | female  | male
Demetra  | female  | male
Glyn  | female  | male
Cleva  | female  | male
Heather  | female  | male
Clementine  | female  | male
Willyt  | female  | male
Jaclin  | female  | male
Pierrette  | female  | male
Matti  | female  | male
Marketa  | female  | male
Auberta  | female  | male
Shalne  | female  | male
Wilma  | female  | male
Grazia  | female  | male
Charmain  | female  | male
Nedda  | female  | male
Charmaine  | female  | male
Mavis  | female  | male
Germain  | female  | male
Debera  | female  | male
Julee  | female  | male
Alla  | female  | male
Demeter  | female  | male
Adena  | female  | male
Constantina  | female  | male
Oreste  | female  | male
Maura  | fe

Можно предопложить, что проблема с некоторыми именами вроде Phillis, Emanuela и Pierrette проблема заключается в том, что а обучающей выборке было больше мужских имён с таким началом.

Проверим нашу гипотезу.

In [101]:
for i, n in enumerate(x_train):
    if n.startswith('Phill'):
        if y_train[i] == 'male':
            print(n + ' m')
        else:
            print(n + ' f')

Phillida f
Phillie f
Philly f
Phillipe m
Phillipe m
Phillipp m
Phillipp m
Phillip m


In [106]:
for i, n in enumerate(x_train):
    if n.startswith('Eman'):
        if y_train[i] == 'male':
            print(n + ' m')
        else:
            print(n + ' f')

Emanuel m


In [105]:
for i, n in enumerate(x_train):
    if n.startswith('Pier'):
        if y_train[i] == 'male':
            print(n + ' m')
        else:
            print(n + ' f')

Pierette f
Pier f
Pierson m
Pierre m
Pierre m


##  Часть 3. Нейросетевая 

### Нейронная сеть [5 баллов]


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$ – то есть, используется one hot encoding.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor

#### делаем one-hot-encoded данные

In [12]:
alphabet = sorted(set(''.join(df['name'])))

In [13]:
maxlen = max(df['name'].map(lambda x: len(x)))
maxlen

15

In [14]:
def one_hot(data, letters, maxlen):
    '''np.array вида (кол-во примеров х бинарная матрица размера (количество букв в алфавите × максимальная длина имени)'''
    res = np.zeros((len(data), maxlen, len(letters)))
    for i in range(len(data)): # по именам
        for l in range(len(data[i])): # по буквам
            res[i][l][letters.index(data[i][l])] = 1
    return res

In [15]:
X_train = one_hot(x_train, alphabet, maxlen)
y_train = np.array(list(map(lambda x: (1, 0) if x=='female' else (0, 1), y_train)))
X_train.shape

(7416, 15, 55)

#### а теперь собственно модели

In [16]:
from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout
from keras.utils import to_categorical

Using TensorFlow backend.


In [17]:
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.output_shape

(None, 2)

In [19]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train, y_train, batch_size=len(X_train), epochs=1, verbose=0)

In [20]:
X_test = one_hot(x_test, alphabet, maxlen)
y_test = to_categorical(np.array(list(map(lambda x: 0 if x=='female' else 1, y_test))), num_classes=2)
preds = model.predict(X_test, verbose=0)

In [21]:
preds = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds)))

In [22]:
print(classification_report(y_test, preds))
print(accuracy_score(y_test, preds))

             precision    recall  f1-score   support

          0       0.79      0.82      0.80       928
          1       0.81      0.78      0.79       928

avg / total       0.80      0.80      0.80      1856

0.796875


In [211]:
# другие методы
# добавляем дропаут
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds = model.predict(X_test, verbose=0)
preds = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds))
print(accuracy_score(y_test, preds))

             precision    recall  f1-score   support

          0       0.78      0.81      0.80       928
          1       0.81      0.77      0.79       928

avg / total       0.79      0.79      0.79      1856

0.7914870689655172


In [212]:
# меняем значение дропаута
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds2 = model.predict(X_test, verbose=0)
preds2 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds2)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds2))
print(accuracy_score(y_test, preds2))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80       928
          1       0.80      0.80      0.80       928

avg / total       0.80      0.80      0.80      1856

0.8033405172413793


In [214]:
# один LSTM-слой
model = Sequential()
model.add(LSTM(128, return_sequences = False, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds3 = model.predict(X_test, verbose=0)
preds3 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds3)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds3))
print(accuracy_score(y_test, preds3))

             precision    recall  f1-score   support

          0       0.77      0.82      0.79       928
          1       0.80      0.75      0.78       928

avg / total       0.79      0.79      0.79      1856

0.7855603448275862


In [23]:
# меняем количество узлов
model = Sequential()
model.add(LSTM(256, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds4 = model.predict(X_test, verbose=0)
preds4 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds4)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds4))
print(accuracy_score(y_test, preds4))

             precision    recall  f1-score   support

          0       0.80      0.82      0.81       928
          1       0.81      0.79      0.80       928

avg / total       0.80      0.80      0.80      1856

0.8033405172413793


In [24]:
# меняем количество слоёв
model = Sequential()
model.add(LSTM(128, return_sequences = True, input_shape=(maxlen, len(alphabet))))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences = False))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose=0)
    
preds5 = model.predict(X_test, verbose=0)
preds5 = np.array(list(map(lambda x: [1., 0.] if x[0]>x[1] else [0., 1.], preds5)))

from sklearn.metrics import classification_report
print(classification_report(y_test, preds5))
print(accuracy_score(y_test, preds5))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80       928
          1       0.80      0.80      0.80       928

avg / total       0.80      0.80      0.80      1856

0.7990301724137931


Итог: ничего не меняется! Только лучше два LSTM-слоя, чем один. 

In [28]:
for i in range(len(y_test)):
    if all(y_test[i] != preds5[i]):
        print(x_test[i], y_test[i])

Elsbeth [1. 0.]
Oreste [1. 0.]
Jonell [1. 0.]
Violet [1. 0.]
Margery [1. 0.]
Shel [1. 0.]
Ingaborg [1. 0.]
Nicol [1. 0.]
Deerdre [1. 0.]
Hali [1. 0.]
Venus [1. 0.]
Jenifer [1. 0.]
Orel [1. 0.]
Rakel [1. 0.]
Deirdre [1. 0.]
Wanda [1. 0.]
Taryn [1. 0.]
Nance [1. 0.]
Merci [1. 0.]
Prudence [1. 0.]
Becky [1. 0.]
Beilul [1. 0.]
Bekki [1. 0.]
Idell [1. 0.]
Ines [1. 0.]
Carrol [1. 0.]
Pier [1. 0.]
Katheryn [1. 0.]
Vere [1. 0.]
Burta [1. 0.]
Gates [1. 0.]
Marjy [1. 0.]
Vicky [1. 0.]
Shandy [1. 0.]
Valma [1. 0.]
Nova [1. 0.]
Raven [1. 0.]
Damaris [1. 0.]
Ame [1. 0.]
Mavis [1. 0.]
Mitzi [1. 0.]
Inger [1. 0.]
Brunhilde [1. 0.]
Milicent [1. 0.]
Mommy [1. 0.]
Deloris [1. 0.]
Sukey [1. 0.]
Farand [1. 0.]
Harri [1. 0.]
Greer [1. 0.]
Francesca [1. 0.]
Steffane [1. 0.]
Kimberly [1. 0.]
Audre [1. 0.]
Ardith [1. 0.]
Midge [1. 0.]
Stormi [1. 0.]
Marijo [1. 0.]
Meghan [1. 0.]
Beatriz [1. 0.]
Betty [1. 0.]
Mignon [1. 0.]
Tiff [1. 0.]
Sharl [1. 0.]
Windy [1. 0.]
Sileas [1. 0.]
Cher [1. 0.]
Trudey [1. 0.]
Sto

##  Часть 4. Сомнительная 

### Визуализация [5 баллов]

Теперь будем использовать другое представление имени: замените каждый символ на его id. Теперь каждое имя – это последовательность длины maxlen. Вы будете обучать эмбеддинги для каждого символа, для этого добавьте в нейронную сети слой Embedding(trainable = True). Как изменение представления символа повлияет на качество классификации?


Побочным продуктом обучения нейронной сети будут эмбеддинги символов – 26 векторов для всех букв латинского алфавита (+ другие символы, которые вы посчитали нужным использовать). Попробуйте использовать разные методы снижения размерности – SVD или TSNE – и изобразить точки, соответствующие разным символам, на плоскости. Получаются ли осмысленные и интерпретируемые кластеры?